In [6]:
import pandas as pd
import os

df = pd.read_csv(os.path.dirname(os.getcwd())+'\data\winemag-data-130k-v2.csv')
df.rename( columns={'Unnamed: 0':'id'}, inplace=True )
df

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss


In [7]:
df=df.dropna(subset=['points','province','variety'])

In [8]:
# country points province varity
import numpy as np
coun = df['country'].to_numpy(dtype='str')
provinces = df['province'].to_numpy(dtype='str')
desig = df['designation'].to_numpy(dtype='str')
reg1 = df['region_1'].to_numpy(dtype='str')
var = df['variety'].to_numpy(dtype='str')
winery = df['winery'].to_numpy(dtype='str')
print('unique countries: ',len(np.unique(coun)))
print('unique provinces: ',len(np.unique(provinces)))
print('unique designations: ',len(np.unique(desig)))
print('unique regions: ',len(np.unique(reg1)))
print('unique varieties: ',len(np.unique(var)))
print('unique wineries: ',len(np.unique(winery)))


unique countries:  43
unique provinces:  425
unique designations:  37955
unique regions:  1230
unique varieties:  701
unique wineries:  16745


In [9]:
dataset = df[['province','variety','points','price']]

In [10]:
dataset.head()

,province,variety,points,price
0,Sicily & Sardinia,White Blend,87,NaN
1,Douro,Portuguese Red,87,15.0
2,Oregon,Pinot Gris,87,14.0
3,Michigan,Riesling,87,13.0
4,Oregon,Pinot Noir,87,65.0


In [11]:
test_data = dataset[dataset['price'].isna()]
test_data = test_data[['province','variety','points']]

train_data = dataset[dataset['price'].notna()]
y = train_data[['price']]
train_data = train_data[['province','variety','points']]
print('test data:',len(test_data))
print('train data:',len(train_data))

test data: 8992
train data: 120915


In [12]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

In [13]:
enc.fit(pd.concat([train_data,test_data]))
X_train=enc.transform(train_data).toarray()
X_test=enc.transform(test_data).toarray()

In [14]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y, test_size=0.20, random_state=42, shuffle=False)

In [15]:
X_train[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [16]:
print(enc.categories_)

[array(['Achaia', 'Aconcagua Costa', 'Aconcagua Valley', 'Aegean',
       'Agioritikos', 'Ahr', 'Alenquer', 'Alentejano', 'Alentejo',
       'Alenteo', 'Algarve', 'Alsace', 'America', 'Amindeo', 'Amyndeon',
       'Andalucia', 'Ankara', 'Apalta', 'Arcadia', 'Arizona', 'Armenia',
       'Atalanti Valley', 'Atlantida', 'Attica', 'Australia Other',
       'Austria', 'Awatere Valley', 'Baden', 'Bairrada', 'Beaujolais',
       'Beira Atlantico', 'Beira Interior', 'Beiras', 'Bekaa Valley',
       'Beotia', 'Black Sea Coastal', 'Bordeaux', 'Bot River', 'Brazil',
       'Brda', 'Breede River Valley', 'Breedekloof', 'British Columbia',
       'Bucelas', 'Buin', 'Bulgaria', 'Burgenland', 'Burgundy',
       'Bío Bío Valley', 'Cachapoal Valley', 'Cahul', 'California',
       'Campanha', 'Canada Other', 'Canelones', 'Canterbury',
       'Cape Agulhas', 'Cape Peninsula', 'Cape South Coast', 'Cappadocia',
       'Carnuntum', 'Casablanca & Leyda Valleys', 'Casablanca Valley',
       'Casablanca-Curicó

In [17]:

data = [enc.categories_[0].tolist(),enc.categories_[1].tolist(),enc.categories_[2].tolist()]
categ = pd.DataFrame(data)
categ.to_csv ('categories.csv', index = False, header=False, encoding='utf-8')

In [18]:
print(X_train[0])
train_data

[0. 0. 0. ... 0. 0. 0.]


,province,variety,points
1,Douro,Portuguese Red,87
2,Oregon,Pinot Gris,87
3,Michigan,Riesling,87
4,Oregon,Pinot Noir,87
5,Northern Spain,Tempranillo-Merlot,87
...,...,...,...
129966,Mosel,Riesling,90
129967,Oregon,Pinot Noir,90
129968,Alsace,Gewürztraminer,90
129969,Alsace,Pinot Gris,90


In [19]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


In [20]:
X_train[0].shape

(1147,)

In [21]:
model = keras.Sequential([
    layers.Dense(1024, activation='relu'),
#     layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])


model.compile(loss='mean_absolute_error',optimizer='adam')

In [ ]:
history = model.fit(X_train,y_train,validation_data=(X_valid,y_valid),verbose=1, epochs=10)

Epoch 1/10
3023/3023 [==============================] - 34s 11ms/step - loss: 13.2533 - val_loss: 12.7039
Epoch 2/10
3023/3023 [==============================] - 33s 11ms/step - loss: 12.1423 - val_loss: 12.5023
Epoch 3/10
3023/3023 [==============================] - 34s 11ms/step - loss: 11.9526 - val_loss: 12.4174
Epoch 4/10
3023/3023 [==============================] - 34s 11ms/step - loss: 11.8287 - val_loss: 12.4391
Epoch 5/10
3023/3023 [==============================] - 33s 11ms/step - loss: 11.7484 - val_loss: 12.3992
Epoch 6/10
3023/3023 [==============================] - 33s 11ms/step - loss: 11.6804 - val_loss: 12.3461
Epoch 7/10
3023/3023 [==============================] - 33s 11ms/step - loss: 11.6201 - val_loss: 12.3304
Epoch 8/10
1926/3023 [==================>...........] - ETA: 11s - loss: 11.6034

In [ ]:
import matplotlib.pyplot as plt
def plot_loss(history):
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error')
    plt.legend()
    plt.grid(True)

plot_loss(history)

In [ ]:
model.save_weights('saved_model')

In [ ]:
test = np.zeroes(1147)

In [ ]:
model.predict()